In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import random
import os
from tqdm import tqdm
import warnings

# ---------------------------
# 1. Configuration Parameters
# ---------------------------
N_SAVED_MODELS = 5000       # Number of memorized models to save
SUBSET_SIZE = 100          # Number of examples per subset
HELD_OUT_SIZE = 10        # 10% of 60,000 FashionMNIST training data
SAVE_FOLDER = './saved_examples'  # Directory to save data-model pairs

# Early stopping parameters
TOLERANCE = 1e-4            # Loss threshold for memorization
PATIENCE = 10               # Number of epochs with no improvement after which training stops
DELTA = 1e-5                # Minimum change in loss to qualify as an improvement
MAX_EPOCHS = 1000           # Maximum number of training epochs

# ---------------------------
# 2. Set Random Seeds
# ---------------------------
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# ---------------------------
# 3. Device Configuration
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ---------------------------
# 4. Data Preparation Functions
# ---------------------------
def load_fashion_mnist():
    """
    Loads the FashionMNIST training dataset.
    """
    transform = transforms.Compose([transforms.ToTensor()])
    train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    return train_dataset

def create_held_out_set(train_dataset, held_out_size=HELD_OUT_SIZE):
    """
    Splits the training dataset into a held-out set and a reduced training set.
    
    Args:
        train_dataset (Dataset): The full FashionMNIST training dataset.
        held_out_size (int): Number of examples to hold out for testing.
    
    Returns:
        held_out_data (Tensor): Held-out images.
        held_out_targets (Tensor): Held-out labels.
        train_indices (list): Indices for the reduced training set.
    """
    total_train = len(train_dataset)
    if held_out_size > total_train:
        raise ValueError("Held-out size exceeds the total training data size.")
    
    indices = list(range(total_train))
    random.shuffle(indices)
    
    held_out_indices = indices[:held_out_size]
    train_indices = indices[held_out_size:]
    
    held_out_data = torch.stack([train_dataset[i][0].view(-1) for i in held_out_indices]).to(device)
    held_out_targets = torch.tensor([train_dataset[i][1] for i in held_out_indices], dtype=torch.long).to(device)
    
    return held_out_data, held_out_targets, train_indices

def sample_subset(train_dataset, train_indices, subset_size=SUBSET_SIZE):
    """
    Randomly samples a subset of the specified size from the training dataset.
    Allows overlapping across subsets.
    
    Args:
        train_dataset (Dataset): The reduced FashionMNIST training dataset.
        train_indices (list): Indices for the reduced training set.
        subset_size (int): Number of examples per subset.
    
    Returns:
        subset_data (Tensor): Subset data.
        subset_targets (Tensor): Subset targets.
    """
    total_train = len(train_indices)
    if subset_size > total_train:
        warnings.warn(f"Requested subset size ({subset_size}) exceeds the available training data size ({total_train}). Capping to {total_train}.")
        subset_size = total_train
    
    subset_indices = random.sample(train_indices, subset_size)
    subset_data = torch.stack([train_dataset[i][0].view(-1) for i in subset_indices]).to(device)
    subset_targets = torch.tensor([train_dataset[i][1] for i in subset_indices], dtype=torch.long).to(device)
    
    return subset_data, subset_targets

# ---------------------------
# 5. Model Definition
# ---------------------------
class SimpleNN(nn.Module):
    """
    Defines a simple neural network with a variable number of layers.
    For this experiment, we'll use a 1-layer model.
    """
    def __init__(self, input_size=784, hidden_size=8, num_layers=1, output_size=10):
        super(SimpleNN, self).__init__()
        layers = []
        current_size = input_size
        for _ in range(num_layers):
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(nn.ReLU())
            current_size = hidden_size
        layers.append(nn.Linear(current_size, output_size))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# ---------------------------
# 6. Training and Evaluation Functions
# ---------------------------
def train_until_memorization(model, data, targets, criterion, optimizer, 
                             tolerance=TOLERANCE, patience=PATIENCE, delta=DELTA, max_epochs=MAX_EPOCHS):
    """
    Trains the model until it perfectly memorizes the data or early stopping criteria are met.
    
    Args:
        model (nn.Module): The neural network to train.
        data (Tensor): Input data.
        targets (Tensor): Target labels.
        criterion (Loss): Loss function.
        optimizer (Optimizer): Optimization algorithm.
        tolerance (float): Loss threshold for memorization.
        patience (int): Number of epochs with no improvement after which training stops.
        delta (float): Minimum change in loss to qualify as an improvement.
        max_epochs (int): Maximum number of training epochs.
    
    Returns:
        memorized (bool): Whether the model achieved 100% memorization.
        final_loss (float): Final training loss.
    """
    best_loss = float('inf')
    epochs_no_improve = 0
    
    for epoch in range(max_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        current_loss = loss.item()
        
        # Check for improvement
        if best_loss - current_loss > delta:
            best_loss = current_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        
        # Check for memorization
        if current_loss < tolerance:
            return True, current_loss
        
        # Check for early stopping
        if epochs_no_improve >= patience:
            return False, current_loss
    
    return False, current_loss

def evaluate_memorization(model, data, targets):
    """
    Evaluates whether the model has perfectly memorized the data.
    
    Args:
        model (nn.Module): The trained neural network.
        data (Tensor): Input data.
        targets (Tensor): Target labels.
    
    Returns:
        memorized (bool): Whether the model achieved 100% memorization.
        accuracy (float): Training accuracy.
    """
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == targets).sum().item()
        total = targets.size(0)
        accuracy = correct / total
    return accuracy == 1.0, accuracy

def evaluate_test_set(model, test_data, test_targets):
    """
    Evaluates the model's accuracy on the held-out test set.
    
    Args:
        model (nn.Module): The trained neural network.
        test_data (Tensor): Held-out test data.
        test_targets (Tensor): Held-out test targets.
    
    Returns:
        accuracy (float): Test accuracy.
    """
    model.eval()
    with torch.no_grad():
        outputs = model(test_data)
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == test_targets).sum().item()
        total = test_targets.size(0)
        accuracy = correct / total
    return accuracy

def save_example(folder, example_id, data, targets, model):
    """
    Saves the data and model as a single file in the specified folder.
    
    Args:
        folder (str): Directory to save the examples.
        example_id (int): Unique identifier for the example.
        data (Tensor): Subset data.
        targets (Tensor): Subset targets.
        model (nn.Module): Trained model.
    """
    if not os.path.exists(folder):
        os.makedirs(folder)
    save_path = os.path.join(folder, f"example_{example_id}.pt")
    torch.save({
        'data': data.cpu(),
        'targets': targets.cpu(),
        'model_state_dict': model.state_dict(),
    }, save_path)

# ---------------------------
# 7. Experiment Runner
# ---------------------------
def run_experiment(train_dataset, train_indices, held_out_data, held_out_targets, 
                   n_saved_models=N_SAVED_MODELS, subset_size=SUBSET_SIZE, save_folder=SAVE_FOLDER):
    """
    Runs the training experiment until the desired number of memorized models are saved.
    
    Args:
        train_dataset (Dataset): The reduced FashionMNIST training dataset.
        train_indices (list): Indices for the reduced training set.
        held_out_data (Tensor): Held-out test data.
        held_out_targets (Tensor): Held-out test targets.
        n_saved_models (int): Number of memorized models to save.
        subset_size (int): Number of examples per subset.
        save_folder (str): Directory to save the examples.
    
    Returns:
        saved_count (int): Number of successfully saved models.
    """
    saved_count = 0
    attempt_count = 0
    max_attempts = n_saved_models * 10  # To prevent infinite loops
    
    print(f"Starting training to save {n_saved_models} memorized models...\n")
    
    with tqdm(total=n_saved_models, desc="Saved Models") as pbar:
        while saved_count < n_saved_models and attempt_count < max_attempts:
            attempt_count += 1
            # Sample a random subset
            subset_data, subset_targets = sample_subset(train_dataset, train_indices, subset_size=subset_size)
            
            # Initialize model
            model = SimpleNN(num_layers=1).to(device)
            
            # Define loss and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.01)
            
            # Train the model
            memorized, final_loss = train_until_memorization(
                model, subset_data, subset_targets, 
                criterion, optimizer
            )
            
            # If not memorized by training, evaluate
            if not memorized:
                memorized, training_accuracy = evaluate_memorization(model, subset_data, subset_targets)
                if memorized:
                    training_accuracy = 1.0
                else:
                    training_accuracy = training_accuracy
            else:
                training_accuracy = 1.0  # Memorization achieved
            
            # If memorized, save the example
            if memorized:
                saved_count += 1
                save_example(save_folder, saved_count, subset_data, subset_targets, model)
                # Evaluate on held-out test set
                test_accuracy = evaluate_test_set(model, held_out_data, held_out_targets)
                # Print summary
                print(f"Saved {saved_count}/{n_saved_models}: Memorized: {memorized}, "
                      f"Final Loss: {final_loss:.6f}, Training Acc: {training_accuracy*100:.2f}%, "
                      f"Test Acc: {test_accuracy*100:.2f}%")
                pbar.update(1)
        
        if saved_count < n_saved_models:
            print(f"\nReached maximum attempts ({max_attempts}) with {saved_count} saved models.")
        else:
            print(f"\nSuccessfully saved {saved_count} memorized models.")
    
    return saved_count

# ---------------------------
# 8. Main Execution
# ---------------------------
def main():
    # Load FashionMNIST
    print("Loading FashionMNIST dataset...")
    train_dataset = load_fashion_mnist()
    print("FashionMNIST dataset loaded.\n")
    
    # Create held-out test set
    print("Creating held-out test set...")
    held_out_data, held_out_targets, train_indices = create_held_out_set(train_dataset, held_out_size=HELD_OUT_SIZE)
    print(f"Held-out test set created with {HELD_OUT_SIZE} examples.\n")
    
    # Run the experiment
    saved_count = run_experiment(
        train_dataset, 
        train_indices, 
        held_out_data, 
        held_out_targets, 
        n_saved_models=N_SAVED_MODELS, 
        subset_size=SUBSET_SIZE, 
        save_folder=SAVE_FOLDER
    )
    
    print(f"\nExperiment completed. {saved_count} models saved to '{SAVE_FOLDER}'.")

if __name__ == "__main__":
    main()
ChatGPT



Using device: cuda
Loading FashionMNIST dataset...
FashionMNIST dataset loaded.

Creating held-out test set...
Held-out test set created with 10 examples.

Starting training to save 5000 memorized models...



Saved Models:   0%|          | 1/5000 [00:01<1:33:09,  1.12s/it]

Saved 1/5000: Memorized: True, Final Loss: 0.000338, Training Acc: 100.00%, Test Acc: 70.00%


Saved Models:   0%|          | 2/5000 [00:02<1:28:23,  1.06s/it]

Saved 2/5000: Memorized: True, Final Loss: 0.000328, Training Acc: 100.00%, Test Acc: 60.00%


...

Saved 4298/5000: Memorized: True, Final Loss: 0.000326, Training Acc: 100.00%, Test Acc: 60.00%


Saved Models:  86%|████████▌ | 4299/5000 [1:37:25<15:50,  1.36s/it]

Saved 4299/5000: Memorized: True, Final Loss: 0.000345, Training Acc: 100.00%, Test Acc: 50.00%


Saved Models:  86%|████████▌ | 4299/5000 [1:37:26<15:53,  1.36s/it]


KeyboardInterrupt: 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import random
import os
from tqdm import tqdm
import warnings

# ---------------------------
# 1. Configuration Parameters
# ---------------------------
SUBSET_SIZE = 100          # Number of examples per subset
SAVE_FOLDER = './saved_examples'  # Directory to save data-model pairs

# Early stopping parameters
TOLERANCE = 1e-4            # Loss threshold for memorization
PATIENCE = 10               # Number of epochs with no improvement after which training stops
DELTA = 1e-5                # Minimum change in loss to qualify as an improvement
MAX_EPOCHS = 1000           # Maximum number of training epochs

# ---------------------------
# 2. Set Random Seeds
# ---------------------------
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# ---------------------------
# 3. Device Configuration
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ---------------------------
# 4. Data Preparation Functions
# ---------------------------
def load_fashion_mnist():
    """
    Loads the FashionMNIST training and test datasets.
    """
    transform = transforms.Compose([transforms.ToTensor()])
    train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
    return train_dataset, test_dataset

def partition_subsets(train_dataset, subset_size=SUBSET_SIZE):
    """
    Partitions the training dataset into non-overlapping subsets of specified size.

    Args:
        train_dataset (Dataset): The full FashionMNIST training dataset.
        subset_size (int): Number of examples per subset.

    Returns:
        List of tuples, each containing subset_data and subset_targets.
    """
    total_train = len(train_dataset)  # 60,000
    indices = list(range(total_train))
    random.shuffle(indices)  # Shuffle to ensure randomness

    num_full_subsets = total_train // subset_size  # 600
    subsets = []

    for i in range(num_full_subsets):
        subset_indices = indices[i * subset_size : (i + 1) * subset_size]
        subset_data = torch.stack([train_dataset[i][0].view(-1) for i in subset_indices]).to(device)
        subset_targets = torch.tensor([train_dataset[i][1] for i in subset_indices], dtype=torch.long).to(device)
        subsets.append((subset_data, subset_targets))

    # Handle the remaining data points if any (should be 0 in this case)
    remaining = total_train % subset_size
    if remaining > 0:
        subset_indices = indices[num_full_subsets * subset_size :]
        subset_data = torch.stack([train_dataset[i][0].view(-1) for i in subset_indices]).to(device)
        subset_targets = torch.tensor([train_dataset[i][1] for i in subset_indices], dtype=torch.long).to(device)
        subsets.append((subset_data, subset_targets))
        print(f"Note: Added a smaller subset with {remaining} samples.")

    return subsets

# ---------------------------
# 5. Model Definition
# ---------------------------
class SimpleNN(nn.Module):
    """
    Defines a simple neural network with a variable number of layers.
    For this experiment, we'll use a 1-layer model.
    """
    def __init__(self, input_size=784, hidden_size=8, num_layers=1, output_size=10):
        super(SimpleNN, self).__init__()
        layers = []
        current_size = input_size
        for _ in range(num_layers):
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(nn.ReLU())
            current_size = hidden_size
        layers.append(nn.Linear(current_size, output_size))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# ---------------------------
# 6. Training and Evaluation Functions
# ---------------------------
def train_until_memorization(model, data, targets, criterion, optimizer, 
                             tolerance=TOLERANCE, patience=PATIENCE, delta=DELTA, max_epochs=MAX_EPOCHS):
    """
    Trains the model until it perfectly memorizes the data or early stopping criteria are met.

    Args:
        model (nn.Module): The neural network to train.
        data (Tensor): Input data.
        targets (Tensor): Target labels.
        criterion (Loss): Loss function.
        optimizer (Optimizer): Optimization algorithm.
        tolerance (float): Loss threshold for memorization.
        patience (int): Number of epochs with no improvement after which training stops.
        delta (float): Minimum change in loss to qualify as an improvement.
        max_epochs (int): Maximum number of training epochs.

    Returns:
        memorized (bool): Whether the model achieved 100% memorization.
        final_loss (float): Final training loss.
    """
    best_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(max_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        current_loss = loss.item()

        # Check for improvement
        if best_loss - current_loss > delta:
            best_loss = current_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # Check for memorization
        if current_loss < tolerance:
            return True, current_loss

        # Check for early stopping
        if epochs_no_improve >= patience:
            return False, current_loss

    return False, current_loss

def evaluate_memorization(model, data, targets):
    """
    Evaluates whether the model has perfectly memorized the data.

    Args:
        model (nn.Module): The trained neural network.
        data (Tensor): Input data.
        targets (Tensor): Target labels.

    Returns:
        memorized (bool): Whether the model achieved 100% memorization.
        accuracy (float): Training accuracy.
    """
    model.eval()
    with torch.no_grad():
        outputs = model(data)
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == targets).sum().item()
        total = targets.size(0)
        accuracy = correct / total
    return accuracy == 1.0, accuracy

def evaluate_test_set(model, test_data, test_targets):
    """
    Evaluates the model's accuracy on the standard test set.

    Args:
        model (nn.Module): The trained neural network.
        test_data (Tensor): Test data.
        test_targets (Tensor): Test targets.

    Returns:
        accuracy (float): Test accuracy.
    """
    model.eval()
    with torch.no_grad():
        outputs = model(test_data)
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == test_targets).sum().item()
        total = test_targets.size(0)
        accuracy = correct / total
    return accuracy

def save_example(folder, example_id, data, targets, model):
    """
    Saves the data and model as a single file in the specified folder.

    Args:
        folder (str): Directory to save the examples.
        example_id (int): Unique identifier for the example.
        data (Tensor): Subset data.
        targets (Tensor): Subset targets.
        model (nn.Module): Trained model.
    """
    if not os.path.exists(folder):
        os.makedirs(folder)
    save_path = os.path.join(folder, f"example_{example_id}.pt")
    torch.save({
        'data': data.cpu(),
        'targets': targets.cpu(),
        'model_state_dict': model.state_dict(),
    }, save_path)

# ---------------------------
# 7. Experiment Runner
# ---------------------------
def run_experiment(train_dataset, subsets, test_dataset, 
                   save_folder=SAVE_FOLDER):
    """
    Runs the training experiment on all subsets.

    Args:
        train_dataset (Dataset): The full FashionMNIST training dataset.
        subsets (list): List of (subset_data, subset_targets) tuples.
        test_dataset (Dataset): The standard FashionMNIST test dataset.
        save_folder (str): Directory to save the examples.

    Returns:
        saved_count (int): Number of successfully saved models.
    """
    saved_count = 0
    total_subsets = len(subsets)
    n_saved_models = total_subsets  # 600

    # Prepare test set
    test_data = torch.stack([test_dataset[i][0].view(-1) for i in range(len(test_dataset))]).to(device)
    test_targets = torch.tensor([test_dataset[i][1] for i in range(len(test_dataset))], dtype=torch.long).to(device)

    print(f"Starting training to save {n_saved_models} memorized models...\n")

    with tqdm(total=n_saved_models, desc="Saved Models") as pbar:
        for idx, (subset_data, subset_targets) in enumerate(subsets, 1):
            # Initialize model
            model = SimpleNN(num_layers=1).to(device)

            # Define loss and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.01)

            # Train the model
            memorized, final_loss = train_until_memorization(
                model, subset_data, subset_targets, 
                criterion, optimizer
            )

            # If not memorized by training, evaluate
            if not memorized:
                memorized, training_accuracy = evaluate_memorization(model, subset_data, subset_targets)
                if memorized:
                    training_accuracy = 1.0
                else:
                    training_accuracy = training_accuracy
            else:
                training_accuracy = 1.0  # Memorization achieved

            # If memorized, save the example
            if memorized:
                saved_count += 1
                save_example(save_folder, saved_count, subset_data, subset_targets, model)
                # Evaluate on the standard test set
                test_accuracy = evaluate_test_set(model, test_data, test_targets)
                # Print summary
                print(f"Saved {saved_count}/{n_saved_models}: Memorized: {memorized}, "
                      f"Final Loss: {final_loss:.6f}, Training Acc: {training_accuracy*100:.2f}%, "
                      f"Test Acc: {test_accuracy*100:.2f}%")
                pbar.update(1)
            else:
                # If not memorized, you can choose to handle it differently
                # For now, we'll skip saving and not increment pbar
                print(f"Subset {idx}: Did not memorize. Final Loss: {final_loss:.6f}, "
                      f"Training Acc: {training_accuracy*100:.2f}%")

    print(f"\nExperiment completed. {saved_count}/{n_saved_models} models saved to '{save_folder}'.")

    return saved_count

# ---------------------------
# 8. Main Execution
# ---------------------------
def main():
    # Load FashionMNIST
    print("Loading FashionMNIST dataset...")
    train_dataset, test_dataset = load_fashion_mnist()
    print("FashionMNIST dataset loaded.\n")

    # Partition the training indices into non-overlapping subsets
    print("Partitioning training data into non-overlapping subsets...")
    subsets = partition_subsets(train_dataset, subset_size=SUBSET_SIZE)
    total_possible_subsets = len(subsets)
    print(f"Partitioned into {total_possible_subsets} subsets of size {SUBSET_SIZE}.\n")

    # Run the experiment
    saved_count = run_experiment(
        train_dataset, 
        subsets, 
        test_dataset, 
        save_folder=SAVE_FOLDER
    )

    print(f"\nExperiment completed. {saved_count}/{total_possible_subsets} models saved to '{SAVE_FOLDER}'.")

if __name__ == "__main__":
    main()


Using device: cuda
Loading FashionMNIST dataset...
FashionMNIST dataset loaded.

Partitioning training data into non-overlapping subsets...
Partitioned into 600 subsets of size 100.

Starting training to save 600 memorized models...



Saved Models:   0%|          | 1/600 [00:01<11:54,  1.19s/it]

Saved 1/600: Memorized: True, Final Loss: 0.000354, Training Acc: 100.00%, Test Acc: 64.13%


Saved Models:   0%|          | 2/600 [00:02<14:40,  1.47s/it]

Saved 2/600: Memorized: True, Final Loss: 0.006085, Training Acc: 100.00%, Test Acc: 58.24%


...

Saved 545/600: Memorized: True, Final Loss: 0.000366, Training Acc: 100.00%, Test Acc: 67.30%


Saved Models:  91%|█████████ | 546/600 [12:24<01:06,  1.23s/it]

Saved 546/600: Memorized: True, Final Loss: 0.000374, Training Acc: 100.00%, Test Acc: 68.62%


Saved Models:  91%|█████████ | 547/600 [12:25<01:12,  1.36s/it]

Saved 547/600: Memorized: True, Final Loss: 0.000344, Training Acc: 100.00%, Test Acc: 63.94%

Experiment completed. 547/600 models saved to './saved_examples'.

Experiment completed. 547/600 models saved to './saved_examples'.


In [ ]:
####################################################################################################################################################################################################################################################################################################################################################################################################################################
################################################################################
####################################################################################################################################################################################################################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################
################################################################################

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import random
from tqdm import tqdm

# ---------------------------
# 1. Configuration Parameters
# ---------------------------
DATA_FOLDER = './saved_examples'   # Folder containing the data-model pairs
TRAIN_RATIO = 0.9                  # Ratio of data to use for training
BATCH_SIZE = 1                     # Number of datasets to process
NUM_EPOCHS = 30                    # Increased number of training epochs for better learning
LEARNING_RATE = 1e-4               # Learning rate for the optimizer
INPUT_SIZE = 784                   # Input size (28x28 images flattened)
HIDDEN_SIZE = 8                    # Hidden size in the target models
OUTPUT_SIZE = 10                   # Number of classes
SUBSET_SIZE = 100                  # Number of examples per subset
PER_EXAMPLE_INPUT_SIZE = INPUT_SIZE + OUTPUT_SIZE  # 784 + 10 = 794
TOTAL_INPUT_SIZE = SUBSET_SIZE * PER_EXAMPLE_INPUT_SIZE  # 100 * 794 = 79,400
PARAM_SIZE = (INPUT_SIZE * HIDDEN_SIZE + HIDDEN_SIZE) + (HIDDEN_SIZE * OUTPUT_SIZE + OUTPUT_SIZE)  # 6370

# ---------------------------
# 2. Set Random Seeds
# ---------------------------
torch.manual_seed(42)
random.seed(42)

# ---------------------------
# 3. Device Configuration
# ---------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ---------------------------
# 4. Model Definitions
# ---------------------------

class SimpleNN(nn.Module):
    def __init__(self, input_size=784, hidden_size=8, num_layers=1, output_size=10):
        super(SimpleNN, self).__init__()
        layers = []
        current_size = input_size
        for _ in range(num_layers):
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(nn.ReLU())
            current_size = hidden_size
        layers.append(nn.Linear(current_size, output_size))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

class Hypernetwork(nn.Module):
    def __init__(self, input_size=79400, output_size=6370, reduction_factor=0.5):
        super(Hypernetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, int(32768 * reduction_factor)),  # Adjusted by reduction factor
            nn.ReLU(),
            nn.Linear(int(32768 * reduction_factor), int(16384 * reduction_factor)),  # Adjusted by reduction factor
            nn.ReLU(),
            nn.Linear(int(16384 * reduction_factor), int(8192 * reduction_factor)),   # Adjusted by reduction factor
            nn.ReLU(),
            nn.Linear(int(8192 * reduction_factor), int(4096 * reduction_factor)),   # Adjusted by reduction factor
            nn.ReLU(),
            nn.Linear(int(4096 * reduction_factor), int(2048 * reduction_factor)),   # Adjusted by reduction factor
            nn.ReLU(),
            nn.Linear(int(2048 * reduction_factor), output_size)
        )

    
    def forward(self, x):
        return self.network(x)

# ---------------------------
# 5. Data Loading Functions
# ---------------------------
def load_data(data_folder):
    """
    Loads the data-model pairs from the specified folder.

    Args:
        data_folder (str): Directory containing the saved examples.

    Returns:
        data_list (list): List of tuples (input_vector, param_vector).
    """
    data_list = []
    for file_name in os.listdir(data_folder):
        if file_name.endswith('.pt'):
            file_path = os.path.join(data_folder, file_name)
            checkpoint = torch.load(file_path, map_location=device)
            data = checkpoint['data']   # Tensor of shape (100, 784)
            targets = checkpoint['targets']   # Tensor of shape (100)
            model_state_dict = checkpoint['model_state_dict']  # Dictionary of model parameters

            # Initialize the model and load state_dict
            model = SimpleNN(input_size=784, hidden_size=8, num_layers=1, output_size=10).to(device)
            model.load_state_dict(model_state_dict)
            
            # Flatten model parameters into a single vector
            param_vector = flatten_model_parameters(model)  # Shape: (6370,)

            # Flatten data and labels into a single input vector
            labels_one_hot = torch.nn.functional.one_hot(targets, num_classes=10).float()  # Shape: (100, 10)
            input_vector = torch.cat([data, labels_one_hot], dim=1).view(-1)  # Shape: (100 * 794,) = (79400,)
            
            data_list.append((input_vector, param_vector))
    return data_list

def split_data(data_list, train_ratio=TRAIN_RATIO):
    """
    Splits the data into training and test sets.

    Args:
        data_list (list): List of data-model pairs.
        train_ratio (float): Ratio of data to use for training.

    Returns:
        train_data (list): Training data.
        test_data (list): Test data.
    """
    random.shuffle(data_list)
    split_idx = int(len(data_list) * train_ratio)
    train_data = data_list[:split_idx]
    test_data = data_list[split_idx:]
    return train_data, test_data

def flatten_model_parameters(model):
    """
    Flattens all parameters of the model into a single vector.

    Args:
        model (nn.Module): The neural network model.

    Returns:
        flat_params (Tensor): Flattened parameter vector.
    """
    params = []
    for param in model.parameters():
        params.append(param.view(-1))
    return torch.cat(params)

# ---------------------------
# 6. Training and Evaluation Functions
# ---------------------------
def train_hypernetwork(train_data, hypernetwork, optimizer, mse_criterion, ce_criterion, epoch):
    """
    Trains the hypernetwork for one epoch using a combined loss function.

    Args:
        train_data (list): List of training data-model pairs.
        hypernetwork (nn.Module): Hypernetwork to generate model parameters.
        optimizer (Optimizer): Optimizer for the hypernetwork.
        mse_criterion (Loss): Mean Squared Error loss for parameter regression.
        ce_criterion (Loss): Cross-Entropy loss for model performance.
        epoch (int): Current epoch number.

    Returns:
        avg_loss (float): Average combined loss over the epoch.
        avg_train_acc (float): Average training accuracy over the epoch.
    """
    hypernetwork.train()
    total_loss = 0.0
    total_train_acc = 0.0
    num_batches = len(train_data)
    
    for idx in tqdm(range(len(train_data)), desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        input_vector, param_vector = train_data[idx]
        input_vector = input_vector.to(device).unsqueeze(0)  # Shape: (1, 79400)
        param_vector = param_vector.to(device)               # Shape: (6370,)
        
        # Generate predicted parameters
        predicted_params = hypernetwork(input_vector)        # Shape: (1, 6370)
        predicted_params = predicted_params.squeeze(0)       # Shape: (6370,)
        
        # Initialize a new SimpleNN model and load predicted parameters
        generated_model = SimpleNN(input_size=784, hidden_size=8, num_layers=1, output_size=10).to(device)
        load_parameters_into_model(generated_model, predicted_params)
        
        # Define MSE loss between predicted_params and true params
        mse_loss = mse_criterion(predicted_params, param_vector)
        
        # Extract input data and labels from the input_vector
        input_data = input_vector.view(SUBSET_SIZE, PER_EXAMPLE_INPUT_SIZE)[:, :INPUT_SIZE]  # Shape: (100, 784)
        labels = input_vector.view(SUBSET_SIZE, PER_EXAMPLE_INPUT_SIZE)[:, INPUT_SIZE:].argmax(dim=1)  # Shape: (100,)
        
        # Forward pass through the generated model
        outputs = generated_model(input_data)  # Shape: (100, 10)
        
        # Compute Cross-Entropy loss
        ce_loss = ce_criterion(outputs, labels)
        
        # Combined loss
        combined_loss = mse_loss + ce_loss
        
        # Backpropagation
        optimizer.zero_grad()
        combined_loss.backward()
        optimizer.step()
        
        # Accumulate loss
        total_loss += combined_loss.item()
        
        # Compute training accuracy
        _, predicted_labels = torch.max(outputs, 1)
        train_acc = (predicted_labels == labels).sum().item() / labels.size(0)
        total_train_acc += train_acc
    
    avg_loss = total_loss / num_batches
    avg_train_acc = (total_train_acc / num_batches) * 100  # Percentage
    return avg_loss, avg_train_acc

def evaluate_hypernetwork(test_data, hypernetwork, mse_criterion, ce_criterion):
    """
    Evaluates the hypernetwork on the test data.

    Args:
        test_data (list): List of test data-model pairs.
        hypernetwork (nn.Module): Hypernetwork to generate model parameters.
        mse_criterion (Loss): Mean Squared Error loss for parameter regression.
        ce_criterion (Loss): Cross-Entropy loss for model performance.

    Returns:
        avg_test_loss (float): Average combined loss on the test data.
        avg_test_acc (float): Average test accuracy.
    """
    hypernetwork.eval()
    total_loss = 0.0
    total_test_acc = 0.0
    num_batches = len(test_data)
    
    with torch.no_grad():
        for idx in tqdm(range(len(test_data)), desc="Evaluating on Test Data"):
            input_vector, param_vector = test_data[idx]
            input_vector = input_vector.to(device).unsqueeze(0)  # Shape: (1, 79400)
            param_vector = param_vector.to(device)               # Shape: (6370,)
            
            # Generate predicted parameters
            predicted_params = hypernetwork(input_vector)        # Shape: (1, 6370)
            predicted_params = predicted_params.squeeze(0)       # Shape: (6370,)
            
            # Initialize a new SimpleNN model and load predicted parameters
            generated_model = SimpleNN(input_size=784, hidden_size=8, num_layers=1, output_size=10).to(device)
            load_parameters_into_model(generated_model, predicted_params)
            
            # Define MSE loss between predicted_params and true params
            mse_loss = mse_criterion(predicted_params, param_vector)
            
            # Extract input data and labels from the input_vector
            input_data = input_vector.view(SUBSET_SIZE, PER_EXAMPLE_INPUT_SIZE)[:, :INPUT_SIZE]  # Shape: (100, 784)
            labels = input_vector.view(SUBSET_SIZE, PER_EXAMPLE_INPUT_SIZE)[:, INPUT_SIZE:].argmax(dim=1)  # Shape: (100,)
            
            # Forward pass through the generated model
            outputs = generated_model(input_data)  # Shape: (100, 10)
            
            # Compute Cross-Entropy loss
            ce_loss = ce_criterion(outputs, labels)
            
            # Combined loss
            combined_loss = mse_loss + ce_loss
            
            # Accumulate loss
            total_loss += combined_loss.item()
            
            # Compute test accuracy
            _, predicted_labels = torch.max(outputs, 1)
            test_acc = (predicted_labels == labels).sum().item() / labels.size(0)
            total_test_acc += test_acc
    
    avg_test_loss = total_loss / num_batches
    avg_test_acc = (total_test_acc / num_batches) * 100  # Percentage
    
    return avg_test_loss, avg_test_acc

# ---------------------------
# 7. Utility Functions
# ---------------------------
def load_parameters_into_model(model, flat_params):
    """
    Loads a flat parameter vector into the model's parameters.

    Args:
        model (nn.Module): The neural network model.
        flat_params (Tensor): Flat parameter vector.
    """
    current_index = 0
    for param in model.parameters():
        param_size = param.numel()
        param.data.copy_(flat_params[current_index:current_index+param_size].view(param.size()))
        current_index += param_size

# ---------------------------
# 8. Main Execution
# ---------------------------
def main():
    # Load data
    print("Loading data...")
    data_list = load_data(DATA_FOLDER)
    print(f"Total data-model pairs loaded: {len(data_list)}\n")
    
    if len(data_list) == 0:
        print("No data found in the specified DATA_FOLDER. Please ensure that the folder contains .pt files.")
        return
    
    # Split data
    train_data, test_data = split_data(data_list)
    print(f"Training data size: {len(train_data)}, Test data size: {len(test_data)}\n")
    
    # Initialize Hypernetwork
    hypernetwork = Hypernetwork(input_size=TOTAL_INPUT_SIZE, output_size=PARAM_SIZE).to(device)
    
    # Define loss functions and optimizer
    mse_criterion = nn.MSELoss()
    ce_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(hypernetwork.parameters(), lr=LEARNING_RATE)
    
    # Training loop
    for epoch in range(NUM_EPOCHS):
        avg_loss, avg_train_acc = train_hypernetwork(
            train_data, hypernetwork, optimizer, mse_criterion, ce_criterion, epoch
        )
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] - Combined Loss: {avg_loss:.6f}, "
              f"Training Accuracy: {avg_train_acc:.2f}%\n")
    
    # Final Evaluation on Test Data
    final_test_loss, final_test_acc = evaluate_hypernetwork(test_data, hypernetwork, mse_criterion, ce_criterion)
    print(f"Final Evaluation on Test Data - Combined Loss: {final_test_loss:.6f}, "
          f"Test Accuracy: {final_test_acc:.2f}%")
    
    # Save the trained hypernetwork
    save_path = 'trained_hypernetwork.pt'
    torch.save({
        'hypernetwork_state_dict': hypernetwork.state_dict(),
    }, save_path)
    print(f"Hypernetwork training completed and saved to '{save_path}'.")

if __name__ == "__main__":
    main()


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import random
from tqdm import tqdm

# ---------------------------
# 1. Configuration Parameters
# ---------------------------
DATA_FOLDER = './saved_examples'   # Folder containing the data-model pairs
TRAIN_RATIO = 0.9                  # Ratio of data to use for training
NUM_EPOCHS = 30                    # Number of training epochs
LEARNING_RATE = 1e-4               # Learning rate for the optimizer

# Target Model Configuration
INPUT_SIZE = 784                   # Input size (28x28 images flattened)
HIDDEN_SIZE = 8                    # Hidden size in the target models
OUTPUT_SIZE = 10                   # Number of classes
SUBSET_SIZE = 40                   # Number of examples per subset (adjusted from 100 to 40 as per user)

# Calculated Parameters
PER_EXAMPLE_INPUT_SIZE = INPUT_SIZE + OUTPUT_SIZE  # 784 + 10 = 794
TOTAL_INPUT_SIZE = SUBSET_SIZE * PER_EXAMPLE_INPUT_SIZE  # 40 * 794 = 31,760

# Target Model Parameter Size
# SimpleNN: (784 * 8 + 8) + (8 * 10 + 10) = 6,280 + 90 = 6,370
PARAM_SIZE = (INPUT_SIZE * HIDDEN_SIZE + HIDDEN_SIZE) + (HIDDEN_SIZE * OUTPUT_SIZE + OUTPUT_SIZE)  # 6370

# Hypernetwork Configuration
NUM_EXAMPLES = SUBSET_SIZE       # Number of examples per subset
EMBED_DIM = 256                   # Embedding dimension per example
# Note: Adjust embed_dim as needed

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ---------------------------
# 2. Model Definitions
# ---------------------------

class EmbeddingBranch(nn.Module):
    """
    Dedicated embedding branch for each input example.
    """
    def __init__(self, input_size=794, embed_dim=256):
        super(EmbeddingBranch, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.network(x)  # Output shape: (batch_size, embed_dim)

class FusionLayer(nn.Module):
    """
    Fusion layer to combine embeddings from all examples using attention.
    """
    def __init__(self, embed_dim=256, num_examples=40):
        super(FusionLayer, self).__init__()
        self.num_examples = num_examples
        self.attention = nn.MultiheadAttention(embed_dim, num_heads=8, dropout=0.1)
        self.layer_norm = nn.LayerNorm(embed_dim)
        self.feedforward = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, embed_dim)
        )
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, x):
        """
        Args:
            x: Tensor of shape (sequence_length=num_examples, batch_size, embed_dim)
        Returns:
            Tensor of shape (sequence_length=num_examples, batch_size, embed_dim)
        """
        attn_output, _ = self.attention(x, x, x)
        x = self.layer_norm(x + self.dropout(attn_output))
        ff_output = self.feedforward(x)
        x = self.layer_norm(x + self.dropout(ff_output))
        return x  # Shape: same as input

class ParameterGenerator(nn.Module):
    """
    Generates the parameter vector from the fused embeddings.
    """
    def __init__(self, embed_dim=256, param_size=6370):
        super(ParameterGenerator, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(embed_dim, 4096),
            nn.ReLU(),
            nn.Linear(4096, param_size)
        )
    
    def forward(self, x):
        return self.network(x)  # Shape: (batch_size, param_size)

class MultiBranchHypernetwork(nn.Module):
    """
    Hypernetwork with dedicated embedding branches for each input example.
    """
    def __init__(self, num_examples=40, per_example_input_size=794, embed_dim=256, param_size=6370):
        super(MultiBranchHypernetwork, self).__init__()
        self.num_examples = num_examples
        self.per_example_input_size = per_example_input_size
        self.embed_dim = embed_dim
        
        # Create a list of embedding branches, one for each example
        self.embedding_branches = nn.ModuleList([
            EmbeddingBranch(input_size=per_example_input_size, embed_dim=embed_dim) for _ in range(num_examples)
        ])
        
        # Fusion layer to combine all embeddings
        self.fusion_layer = FusionLayer(embed_dim=embed_dim, num_examples=num_examples)
        
        # Parameter generator
        self.param_generator = ParameterGenerator(embed_dim=embed_dim, param_size=param_size)
    
    def forward(self, x):
        """
        Args:
            x: Tensor of shape (batch_size, num_examples * per_example_input_size)
        Returns:
            param_vector: Tensor of shape (batch_size, param_size)
        """
        batch_size = x.size(0)
        # Split the input into individual examples
        x = x.view(batch_size, self.num_examples, self.per_example_input_size)  # Shape: (batch_size, num_examples, per_example_input_size)
        
        # Pass each example through its dedicated embedding branch
        embeddings = []
        for i in range(self.num_examples):
            emb = self.embedding_branches[i](x[:, i, :])  # Shape: (batch_size, embed_dim)
            embeddings.append(emb)
        
        # Stack embeddings to form a sequence for the fusion layer
        embeddings = torch.stack(embeddings, dim=0)  # Shape: (num_examples, batch_size, embed_dim)
        
        # Apply fusion layer (e.g., attention-based)
        fused = self.fusion_layer(embeddings)  # Shape: (num_examples, batch_size, embed_dim)
        
        # Aggregate transformer outputs: mean pooling over the sequence
        aggregated = fused.mean(dim=0)  # Shape: (batch_size, embed_dim)
        
        # Generate parameter vector
        param_vector = self.param_generator(aggregated)  # Shape: (batch_size, param_size)
        
        return param_vector

# ---------------------------
# 3. Data Loading Functions
# ---------------------------
def load_data(data_folder):
    """
    Loads the data-model pairs from the specified folder.

    Args:
        data_folder (str): Directory containing the saved examples.

    Returns:
        data_list (list): List of tuples (input_vector, param_vector).
    """
    data_list = []
    for file_name in os.listdir(data_folder):
        if file_name.endswith('.pt'):
            file_path = os.path.join(data_folder, file_name)
            checkpoint = torch.load(file_path, map_location=device)
            data = checkpoint['data']   # Tensor of shape (40, 784)
            targets = checkpoint['targets']   # Tensor of shape (40,)
            model_state_dict = checkpoint['model_state_dict']  # Dictionary of model parameters

            # Initialize the model and load state_dict
            model = SimpleNN(input_size=784, hidden_size=8, num_layers=1, output_size=10).to(device)
            try:
                model.load_state_dict(model_state_dict)
            except RuntimeError as e:
                print(f"Error loading state_dict for file {file_name}: {e}")
                continue  # Skip this file if there's an error

            # Flatten model parameters into a single vector
            param_vector = flatten_model_parameters(model)  # Shape: (6370,)

            # Flatten data and labels into a single input vector
            labels_one_hot = torch.nn.functional.one_hot(targets, num_classes=10).float()  # Shape: (40, 10)
            input_vector = torch.cat([data, labels_one_hot], dim=1).view(-1)  # Shape: (40 * 794,) = (31,760,)

            data_list.append((input_vector, param_vector))
    return data_list

def split_data(data_list, train_ratio=TRAIN_RATIO):
    """
    Splits the data into training and test sets.

    Args:
        data_list (list): List of data-model pairs.
        train_ratio (float): Ratio of data to use for training.

    Returns:
        train_data (list): Training data.
        test_data (list): Test data.
    """
    random.shuffle(data_list)
    split_idx = int(len(data_list) * train_ratio)
    train_data = data_list[:split_idx]
    test_data = data_list[split_idx:]
    return train_data, test_data

def flatten_model_parameters(model):
    """
    Flattens all parameters of the model into a single vector.

    Args:
        model (nn.Module): The neural network model.

    Returns:
        flat_params (Tensor): Flattened parameter vector.
    """
    params = []
    for param in model.parameters():
        params.append(param.view(-1))
    return torch.cat(params)

# ---------------------------
# 4. Training and Evaluation Functions
# ---------------------------
def train_hypernetwork(train_data, hypernetwork, optimizer, mse_criterion, ce_criterion, epoch):
    """
    Trains the hypernetwork for one epoch using a combined loss function.

    Args:
        train_data (list): List of training data-model pairs.
        hypernetwork (nn.Module): Hypernetwork to generate model parameters.
        optimizer (Optimizer): Optimizer for the hypernetwork.
        mse_criterion (Loss): Mean Squared Error loss for parameter regression.
        ce_criterion (Loss): Cross-Entropy loss for model performance.
        epoch (int): Current epoch number.

    Returns:
        avg_loss (float): Average combined loss over the epoch.
        avg_train_acc (float): Average training accuracy over the epoch.
    """
    hypernetwork.train()
    total_loss = 0.0
    total_train_acc = 0.0
    num_batches = len(train_data)
    
    for idx in tqdm(range(len(train_data)), desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        input_vector, param_vector = train_data[idx]
        input_vector = input_vector.to(device).unsqueeze(0)  # Shape: (1, 31,760)
        param_vector = param_vector.to(device)               # Shape: (6370,)
        
        # Generate predicted parameters
        predicted_params = hypernetwork(input_vector)        # Shape: (1, 6370)
        predicted_params = predicted_params.squeeze(0)       # Shape: (6370,)
        
        # Initialize a new SimpleNN model and load predicted parameters
        generated_model = SimpleNN(input_size=784, hidden_size=8, num_layers=1, output_size=10).to(device)
        load_parameters_into_model(generated_model, predicted_params)
        
        # Define MSE loss between predicted_params and true params
        mse_loss = mse_criterion(predicted_params, param_vector)
        
        # Extract input data and labels from the input_vector
        input_data = input_vector.view(SUBSET_SIZE, PER_EXAMPLE_INPUT_SIZE)[:, :INPUT_SIZE]  # Shape: (40, 784)
        labels = input_vector.view(SUBSET_SIZE, PER_EXAMPLE_INPUT_SIZE)[:, INPUT_SIZE:].argmax(dim=1)  # Shape: (40,)
        
        # Forward pass through the generated model
        outputs = generated_model(input_data)  # Shape: (40, 10)
        
        # Compute Cross-Entropy loss
        ce_loss = ce_criterion(outputs, labels)
        
        # Combined loss
        combined_loss = mse_loss + ce_loss
        
        # Backpropagation
        optimizer.zero_grad()
        combined_loss.backward()
        optimizer.step()
        
        # Accumulate loss
        total_loss += combined_loss.item()
        
        # Compute training accuracy
        _, predicted_labels = torch.max(outputs, 1)
        train_acc = (predicted_labels == labels).sum().item() / labels.size(0)
        total_train_acc += train_acc
    
    avg_loss = total_loss / num_batches
    avg_train_acc = (total_train_acc / num_batches) * 100  # Percentage
    return avg_loss, avg_train_acc

def evaluate_hypernetwork(test_data, hypernetwork, mse_criterion, ce_criterion):
    """
    Evaluates the hypernetwork on the test data.

    Args:
        test_data (list): List of test data-model pairs.
        hypernetwork (nn.Module): Hypernetwork to generate model parameters.
        mse_criterion (Loss): Mean Squared Error loss for parameter regression.
        ce_criterion (Loss): Cross-Entropy loss for model performance.

    Returns:
        avg_test_loss (float): Average combined loss on the test data.
        avg_test_acc (float): Average test accuracy.
    """
    hypernetwork.eval()
    total_loss = 0.0
    total_test_acc = 0.0
    num_batches = len(test_data)
    
    with torch.no_grad():
        for idx in tqdm(range(len(test_data)), desc="Evaluating on Test Data"):
            input_vector, param_vector = test_data[idx]
            input_vector = input_vector.to(device).unsqueeze(0)  # Shape: (1, 31,760)
            param_vector = param_vector.to(device)               # Shape: (6370,)
            
            # Generate predicted parameters
            predicted_params = hypernetwork(input_vector)        # Shape: (1, 6370)
            predicted_params = predicted_params.squeeze(0)       # Shape: (6370,)
            
            # Initialize a new SimpleNN model and load predicted parameters
            generated_model = SimpleNN(input_size=784, hidden_size=8, num_layers=1, output_size=10).to(device)
            load_parameters_into_model(generated_model, predicted_params)
            
            # Define MSE loss between predicted_params and true params
            mse_loss = mse_criterion(predicted_params, param_vector)
            
            # Extract input data and labels from the input_vector
            input_data = input_vector.view(SUBSET_SIZE, PER_EXAMPLE_INPUT_SIZE)[:, :INPUT_SIZE]  # Shape: (40, 784)
            labels = input_vector.view(SUBSET_SIZE, PER_EXAMPLE_INPUT_SIZE)[:, INPUT_SIZE:].argmax(dim=1)  # Shape: (40,)
            
            # Forward pass through the generated model
            outputs = generated_model(input_data)  # Shape: (40, 10)
            
            # Compute Cross-Entropy loss
            ce_loss = ce_criterion(outputs, labels)
            
            # Combined loss
            combined_loss = mse_loss + ce_loss
            
            # Accumulate loss
            total_loss += combined_loss.item()
            
            # Compute test accuracy
            _, predicted_labels = torch.max(outputs, 1)
            test_acc = (predicted_labels == labels).sum().item() / labels.size(0)
            total_test_acc += test_acc
    
    avg_test_loss = total_loss / num_batches
    avg_test_acc = (total_test_acc / num_batches) * 100  # Percentage
    
    return avg_test_loss, avg_test_acc

# ---------------------------
# 5. Utility Functions
# ---------------------------
def load_parameters_into_model(model, flat_params):
    """
    Loads a flat parameter vector into the model's parameters.

    Args:
        model (nn.Module): The neural network model.
        flat_params (Tensor): Flat parameter vector.
    """
    current_index = 0
    for param in model.parameters():
        param_size = param.numel()
        param.data.copy_(flat_params[current_index:current_index+param_size].view(param.size()))
        current_index += param_size

# ---------------------------
# 6. Main Execution
# ---------------------------
def main():
    # Load data
    print("Loading data...")
    data_list = load_data(DATA_FOLDER)
    print(f"Total data-model pairs loaded: {len(data_list)}\n")
    
    if len(data_list) == 0:
        print("No data found in the specified DATA_FOLDER. Please ensure that the folder contains .pt files.")
        return
    
    # Split data
    train_data, test_data = split_data(data_list)
    print(f"Training data size: {len(train_data)}, Test data size: {len(test_data)}\n")
    
    # Initialize Hypernetwork
    hypernetwork = MultiBranchHypernetwork(
        num_examples=NUM_EXAMPLES,
        per_example_input_size=PER_EXAMPLE_INPUT_SIZE,
        embed_dim=EMBED_DIM,
        param_size=PARAM_SIZE
    ).to(device)
    
    # Define loss functions and optimizer
    mse_criterion = nn.MSELoss()
    ce_criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(hypernetwork.parameters(), lr=LEARNING_RATE)
    
    # Training loop
    for epoch in range(NUM_EPOCHS):
        avg_loss, avg_train_acc = train_hypernetwork(
            train_data, hypernetwork, optimizer, mse_criterion, ce_criterion, epoch
        )
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] - Combined Loss: {avg_loss:.6f}, "
              f"Training Accuracy: {avg_train_acc:.2f}%\n")
    
    # Final Evaluation on Test Data
    final_test_loss, final_test_acc = evaluate_hypernetwork(test_data, hypernetwork, mse_criterion, ce_criterion)
    print(f"Final Evaluation on Test Data - Combined Loss: {final_test_loss:.6f}, "
          f"Test Accuracy: {final_test_acc:.2f}%")
    
    # Save the trained hypernetwork
    save_path = 'trained_multi_branch_hypernetwork.pt'
    torch.save({
        'hypernetwork_state_dict': hypernetwork.state_dict(),
    }, save_path)
    print(f"Hypernetwork training completed and saved to '{save_path}'.")

if __name__ == "__main__":
    main()


Using device: cuda
Loading data...


NameError: name 'SimpleNN' is not defined